# Interactive Visualization Lab

Complete the following set of exercises to solidify your knowledge of interactive visualization using Plotly, Cufflinks, and IPyWidgets.

In [1]:
import pandas as pd
import chart_studio.plotly as px
import cufflinks as cf
from ipywidgets import interact

cf.go_offline()

In [2]:
data = pd.read_excel('../data/Online Retail.xlsx')

In [3]:
data.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country
0,536365,2010-12-01 08:26:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
1,536373,2010-12-01 09:02:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
2,536375,2010-12-01 09:32:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
3,536390,2010-12-01 10:19:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,64,2.55,163.2,17511,United Kingdom
4,536394,2010-12-01 10:39:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,32,2.55,81.6,13408,United Kingdom


## 1. Create an interactive bar chart showing total quantity and revenue by country (excluding United Kingdom) for the month of April 2011.

In [4]:
data.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country
0,536365,2010-12-01 08:26:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
1,536373,2010-12-01 09:02:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
2,536375,2010-12-01 09:32:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
3,536390,2010-12-01 10:19:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,64,2.55,163.2,17511,United Kingdom
4,536394,2010-12-01 10:39:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,32,2.55,81.6,13408,United Kingdom


In [36]:
data1 = data.copy()

data1['InvoiceDate'] = pd.to_datetime(data1['InvoiceDate'], infer_datetime_format=True)

start_date = "2011-01-01"

end_date = "2011-05-31"

mask = (data1['InvoiceDate'] > start_date) & (data1['InvoiceDate'] <= end_date)

data1 = data1.loc[mask]

data1.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country,Year,Month,Day
207,539993,2011-01-04 10:00:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,12,2.95,35.4,13313,United Kingdom,2011,1,4
208,540014,2011-01-04 11:34:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.95,17.7,14606,United Kingdom,2011,1,4
209,540016,2011-01-04 11:42:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,10,2.95,29.5,16282,United Kingdom,2011,1,4
210,540033,2011-01-04 14:12:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,32,2.55,81.6,13680,United Kingdom,2011,1,4
211,540043,2011-01-04 14:21:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,4,2.95,11.8,15164,United Kingdom,2011,1,4


In [6]:
data1 = data1[["Country","Quantity","Revenue"]]

data1.head()

,Country,Quantity,Revenue
0,United Kingdom,6,15.3
1,United Kingdom,6,15.3
2,United Kingdom,6,15.3
3,United Kingdom,64,163.2
4,United Kingdom,32,81.6


In [40]:
data1 = data1.loc[data["Country"]!="United Kingdom"]

data1.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country,Year,Month,Day
240,540469,2011-01-07 14:04:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.95,17.7,12484,Spain,2011,1,7
304,541570,2011-01-19 12:34:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,256,2.55,652.8,14646,Netherlands,2011,1,19
307,541657,2011-01-20 12:11:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.95,17.7,16321,Australia,2011,1,20
322,541979,2011-01-24 14:54:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.95,17.7,14911,EIRE,2011,1,24
326,542102,2011-01-25 13:26:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,18,2.95,53.1,12744,Singapore,2011,1,25


In [41]:
data1 = data1.groupby('Country').sum().reset_index()

data1.head()

,Country,InvoiceNo,Quantity,UnitPrice,Revenue,CustomerID,Year,Month,Day
0,Australia,249631316,33986,1395.31,54997.13,5704557,919027,1277,5958
1,Austria,59437702,2091,332.23,3380.69,1344564,217188,427,1999
2,Bahrain,9397437,260,78.95,548.40,210027,34187,85,193
3,Belgium,310194993,6612,1653.36,10276.13,7050057,1138226,1867,9227
4,Brazil,17606432,356,142.60,1143.60,408608,64352,128,480


In [42]:
import plotly.graph_objects as go

axisx = data1["Country"]

axisy1 =data1["Quantity"]

axisy2 = data1["Revenue"]

fig = go.Figure(data=[
    go.Bar(name='Revenue', x=axisx, y=axisy1),
    go.Bar(name='Quantity', x=axisx, y=axisy2)
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

## 2. Create an interactive line chart showing quantity and revenue sold to France between January 1st and May 31st 2011.

In [9]:
data.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country
207,539993,2011-01-04 10:00:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,12,2.95,35.4,13313,United Kingdom
208,540014,2011-01-04 11:34:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.95,17.7,14606,United Kingdom
209,540016,2011-01-04 11:42:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,10,2.95,29.5,16282,United Kingdom
210,540033,2011-01-04 14:12:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,32,2.55,81.6,13680,United Kingdom
211,540043,2011-01-04 14:21:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,4,2.95,11.8,15164,United Kingdom


In [10]:
data2 = data.loc[data["Country"]=="France"]

data2 = data2[["InvoiceDate", "Country", "Quantity", "Revenue"]]

data2.head()

,InvoiceDate,Country,Quantity,Revenue
807,2011-04-26 15:52:00,France,9,26.55
2645,2011-01-12 15:00:00,France,4,15.00
3290,2011-02-28 14:45:00,France,4,34.00
4614,2011-01-31 09:57:00,France,160,232.00
4795,2011-03-31 10:27:00,France,160,232.00


In [11]:
data2['InvoiceDate'] = pd.to_datetime(data2['InvoiceDate'], infer_datetime_format=True)

start_date = "2011-01-01"

end_date = "2011-05-31"

mask = (data2['InvoiceDate'] > start_date) & (data2['InvoiceDate'] <= end_date)

data2 = data2.loc[mask]

data2.head()


,InvoiceDate,Country,Quantity,Revenue
807,2011-04-26 15:52:00,France,9,26.55
2645,2011-01-12 15:00:00,France,4,15.00
3290,2011-02-28 14:45:00,France,4,34.00
4614,2011-01-31 09:57:00,France,160,232.00
4795,2011-03-31 10:27:00,France,160,232.00


In [52]:
data2 = data2.groupby('InvoiceDate').sum().reset_index()

import plotly.express as px

import numpy as np

x = data2["InvoiceDate"]


fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=data2["Quantity"],
                    mode='lines',
                    name='Quantity'))
fig.add_trace(go.Scatter(x=x, y=data2["Revenue"],
                    mode='lines',
                    name='Revenue'))

fig.show()



## 3. Create an interactive scatter plot showing the relationship between average quantity (x-axis) and average unit price (y-axis) for the product PARTY BUNTING with the plot points color-coded by country (categories).

In [13]:
data.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country
207,539993,2011-01-04 10:00:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,12,2.95,35.4,13313,United Kingdom
208,540014,2011-01-04 11:34:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.95,17.7,14606,United Kingdom
209,540016,2011-01-04 11:42:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,10,2.95,29.5,16282,United Kingdom
210,540033,2011-01-04 14:12:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,32,2.55,81.6,13680,United Kingdom
211,540043,2011-01-04 14:21:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,4,2.95,11.8,15164,United Kingdom


In [14]:
data3 = data.loc[data["Description"] == "PARTY BUNTING"]

data3 = data3[["Country","Description", "Quantity", "UnitPrice"]]

data3.head()

,Country,Description,Quantity,UnitPrice
252887,United Kingdom,PARTY BUNTING,60,3.75
252888,United Kingdom,PARTY BUNTING,5,4.65
252889,United Kingdom,PARTY BUNTING,50,3.75
252890,United Kingdom,PARTY BUNTING,3,4.65
252891,United Kingdom,PARTY BUNTING,15,4.65


In [15]:
data3 = data3.groupby('Country').agg({"Quantity":"mean", "UnitPrice":"mean"})

data3 = data3.reset_index()

data3.head()

,Country,Quantity,UnitPrice
0,Australia,30.25,4.675
1,Austria,8.00,4.950
2,Belgium,4.00,4.950
3,Cyprus,5.00,4.650
4,EIRE,16.50,4.750


In [58]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data3["Quantity"], y=data3["UnitPrice"],
                    mode='markers',
                    name='Quantity'))
fig.show()

In [53]:
px.scatter(data3, x="Quantity", y="UnitPrice", color="Country")

## 4. Create a set of interactive histograms showing the distributions of quantity per invoice for the following countries: EIRE, Germany, France, and Netherlands.

In [17]:
data.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country
207,539993,2011-01-04 10:00:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,12,2.95,35.4,13313,United Kingdom
208,540014,2011-01-04 11:34:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.95,17.7,14606,United Kingdom
209,540016,2011-01-04 11:42:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,10,2.95,29.5,16282,United Kingdom
210,540033,2011-01-04 14:12:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,32,2.55,81.6,13680,United Kingdom
211,540043,2011-01-04 14:21:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,4,2.95,11.8,15164,United Kingdom


In [18]:
data4 = data[data["Country"].isin(["EIRE","Germany","France","Netherlands"])]

data4 = data4[["Country","InvoiceNo", "Quantity"]]

data4.head()

,Country,InvoiceNo,Quantity
304,Netherlands,541570,256
322,EIRE,541979,6
367,EIRE,542777,6
393,EIRE,543114,32
447,EIRE,544210,6


In [19]:
data4 = pd.pivot_table(data4, index=["Country", "InvoiceNo"], aggfunc="sum").reset_index()

data4.head()

#data4.iplot(kind="histogram", x="InvoiceNo", y="Quantity", categories = "Country")


,Country,InvoiceNo,Quantity
0,EIRE,540547,84
1,EIRE,541220,6198
2,EIRE,541434,286
3,EIRE,541670,1332
4,EIRE,541979,46


In [20]:
cols = data4.columns

data4[cols[0]].iplot(kind='hist', color= 'rgb(0,150,150)')
data4[cols[1]].iplot(kind='hist', color= 'rgb(255,0,0)')
data4[cols[2]].iplot(kind='hist', color= 'rgb(0,255,0)')


In [21]:
fewcountries = data[data['Country'].isin(['EIRE', 'Germany', 'France', 'Netherlands'])]
quantityperinvoice = fewcountries.pivot_table(values='Quantity', columns='Country', 
                      index='InvoiceNo', aggfunc='sum')

quantityperinvoice.head()

Country,EIRE,France,Germany,Netherlands
InvoiceNo,,,,
540178,NaN,359.0,NaN,NaN
540239,NaN,369.0,NaN,NaN
540351,NaN,261.0,NaN,NaN
540365,NaN,177.0,NaN,NaN
540410,NaN,NaN,92.0,NaN


In [22]:
quantityperinvoice.iplot(kind='hist', xTitle='Quantity of Invoice', 
           yTitle='Amount of invoices', title='Distribution of invoice quantities')

## 5. Create an interactive side-by-side bar chart showing the revenue by country listed below (bars) for each of the products listed below.

In [23]:
product_list = ['JUMBO BAG RED RETROSPOT', 
                'CREAM HANGING HEART T-LIGHT HOLDER',
                'REGENCY CAKESTAND 3 TIER']

country_list = ['EIRE', 'Germany', 'France', 'Netherlands']

In [59]:
data.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country,Year,Month,Day
207,539993,2011-01-04 10:00:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,12,2.95,35.4,13313,United Kingdom,2011,1,4
208,540014,2011-01-04 11:34:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.95,17.7,14606,United Kingdom,2011,1,4
209,540016,2011-01-04 11:42:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,10,2.95,29.5,16282,United Kingdom,2011,1,4
210,540033,2011-01-04 14:12:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,32,2.55,81.6,13680,United Kingdom,2011,1,4
211,540043,2011-01-04 14:21:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,4,2.95,11.8,15164,United Kingdom,2011,1,4


In [25]:
data5= data[data['Country'].isin(['EIRE', 'Germany', 'France', 'Netherlands'])]

data5.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country
304,541570,2011-01-19 12:34:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,256,2.55,652.8,14646,Netherlands
322,541979,2011-01-24 14:54:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.95,17.7,14911,EIRE
367,542777,2011-02-01 08:31:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.95,17.7,14911,EIRE
393,543114,2011-02-03 13:26:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,32,2.55,81.6,14156,EIRE
447,544210,2011-02-17 11:01:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.95,17.7,14911,EIRE


In [26]:
data5 = data5[data5['Description'].isin(['JUMBO BAG RED RETROSPOT', 
                'CREAM HANGING HEART T-LIGHT HOLDER',
                'REGENCY CAKESTAND 3 TIER'])]

data5.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country
304,541570,2011-01-19 12:34:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,256,2.55,652.8,14646,Netherlands
322,541979,2011-01-24 14:54:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.95,17.7,14911,EIRE
367,542777,2011-02-01 08:31:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.95,17.7,14911,EIRE
393,543114,2011-02-03 13:26:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,32,2.55,81.6,14156,EIRE
447,544210,2011-02-17 11:01:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.95,17.7,14911,EIRE


In [27]:
data5 = data5.pivot_table(values='Revenue', columns='Country', index='Description', aggfunc='sum')

data5.head()

Country,EIRE,France,Germany,Netherlands
Description,,,,
CREAM HANGING HEART T-LIGHT HOLDER,428.7,26.55,NaN,723.6
JUMBO BAG RED RETROSPOT,63.7,281.45,241.8,1678.0
REGENCY CAKESTAND 3 TIER,2883.0,669.15,3138.0,1576.8


In [28]:
data5.iplot(kind='bar', xTitle='Products', 
           yTitle='Revenue', title='Distribution of invoice quantities')

## 6. Create an interactive line chart showing quantity sold by day for the United Kingdom. Add drop-down boxes for Year and Month that allow you to filter the date range that appears in the chart.

In [29]:
data['Year'] = pd.DatetimeIndex(data['InvoiceDate']).year
data['Month'] = pd.DatetimeIndex(data['InvoiceDate']).month
data['Day'] = pd.DatetimeIndex(data['InvoiceDate']).day
uk = data[data['Country']=='United Kingdom']
data5=uk

In [30]:
data5.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country,Year,Month,Day
207,539993,2011-01-04 10:00:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,12,2.95,35.4,13313,United Kingdom,2011,1,4
208,540014,2011-01-04 11:34:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.95,17.7,14606,United Kingdom,2011,1,4
209,540016,2011-01-04 11:42:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,10,2.95,29.5,16282,United Kingdom,2011,1,4
210,540033,2011-01-04 14:12:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,32,2.55,81.6,13680,United Kingdom,2011,1,4
211,540043,2011-01-04 14:21:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,4,2.95,11.8,15164,United Kingdom,2011,1,4


In [31]:
data5 = data5[["Country", "Year", "Month", "Day", "Quantity"]]

data5.head()

,Country,Year,Month,Day,Quantity
207,United Kingdom,2011,1,4,12
208,United Kingdom,2011,1,4,6
209,United Kingdom,2011,1,4,10
210,United Kingdom,2011,1,4,32
211,United Kingdom,2011,1,4,4


In [32]:
data5 = data5.pivot_table(values='Quantity', columns=['Year','Month'], index='Day', aggfunc='sum')

data5.head()


Year      2011                                    
Month        1        2        3        4        5
Day                                               
1          NaN  13536.0   7614.0  17149.0   3172.0
2          NaN   8184.0   8148.0      NaN      NaN
3          NaN  14502.0   7842.0   5667.0   9566.0
4       6659.0  10226.0  11186.0  10937.0  15624.0
5      17635.0      NaN      NaN  10564.0  13906.0

In [33]:
data5.iplot(kind='line', xTitle='Day', yTitle='Cantidad', title='Cantidades de productos')

## 7. Create an interactive scatter plot that plots number of invoices (x-axis) vs. number of customers (y-axis) and the plot points represent individual products. Add two sliders that control the x and y axis ranges.

In [34]:
agg_func = {'InvoiceNo':'nunique',
            'Quantity':'sum',
            'UnitPrice':'mean',
            'Revenue':'sum',
            'CustomerID':'nunique'}

products = uk.groupby('Description').agg(agg_func)

data5 = products

data5.head()

,InvoiceNo,Quantity,UnitPrice,Revenue,CustomerID
Description,,,,,
4 PURPLE FLOCK DINNER CANDLES,8,14,2.550000,35.70,8
OVAL WALL MIRROR DIAMANTE,25,64,9.710000,492.80,21
SET 2 TEA TOWELS I LOVE LONDON,132,2144,3.053431,5757.20,89
SPACEBOY BABY GIFT SET,35,83,16.561111,1322.85,33
TOADSTOOL BEDSIDE LIGHT,10,18,8.950000,161.10,10


KeyError: "['Description'] not in index"

## 8. Creat an interactive bar chart that shows revenue by product description. Add a text field widget that filters the results to show the product that contain the text entered in their description.